In [ ]:
from networkit import *
import math
%matplotlib inline

In [ ]:
cd "/Users/Henning/Documents/workspace/NetworKit/"

In [ ]:
# deactivate vertex that represents the special part and needs to be deactivated
def deactivateNode(G, x):
    neigh = G.neighbors(x)
    weights = {}

    for u in neigh:
        weights[u] = (G.weight(x, u))
        G.removeEdge(x, u)
    G.removeNode(x)
    
    return G

In [ ]:
# partition G with PLMR and different gamma resolutions
def partition(G, x, Gname):
    n = G.numberOfNodes()
    gammaBy10 = 15 # scaled by 10 due to floating point errors in Python...
    while gammaBy10 >= 5:
        # perform partitioning/clustering
        communities = community.detectCommunities(G, algo=community.PLM(G, True, gammaBy10 / 10))

        # put deactivated vertex back in (as singleton cluster) and connect it to old neighbors
        if (x >= 0):
            communities.toSingleton(x)

        # save communities to file
        path = "output/physchem/" + Gname + "-x=" + str(x) + "-gamma=" + str(gammaBy10 / 10) + ".ptn"
        community.writeCommunities(communities, path)

        ## compute edge cut comparison to continuous solution
        # use number of clusters from PLM solution
        # subtract 1 to account for special vertex
        k = communities.numberOfSubsets() - 1

        # compute continuous partition (and "repair" it)
        avgSize = math.ceil(n / k)   
        contPart = community.ClusteringGenerator().makeSingletonClustering(G)
        for v in G.nodes():
            contPart.moveToSubset(round(math.floor(v / avgSize)), v)
        if (x >= 0):
            contPart.toSingleton(x)

        # modularity score for comparison
        print("Modularity: " + str(community.Modularity().getQuality(contPart, G)))

        # save continuous to file for comparison
        path = "output/physchem/" + Gname + "-contPart-k=" + str(k) + ".ptn"
        community.writeCommunities(contPart, path)

        # print both coverage values
        print("*** COVERAGE COMPARISON ***")
        plmrCov = community.Coverage().getQuality(communities, G)
        contCov = community.Coverage().getQuality(contPart, G)
        print("PLMR: " + str(plmrCov))
        print("CONT: " + str(contCov))

        # infer and print both edge cut values: EC = total edges - internal edges
        tew = G.totalEdgeWeight()
        print("*** EDGE CUT COMPARISON *** total edge weight: ", tew)
        plmrEC = tew * (1.0 - plmrCov)
        contEC = tew * (1.0 - contCov)
        print("PLMR: " + str(plmrEC))
        print("CONT: " + str(contEC))
        if (contEC != 0.0):
            print("!!! Improvement in percent: ", (1 - plmrEC / contEC) * 100, " !!!")
        else:
            print("!!! CONT is perfect!")
        print("\n")

        # set coarseness parameter
        gammaBy10 = gammaBy10 - 2

In [ ]:
# specify graphs to be processed
Gnames = ["bacteriorhodopsin-10-2.5", "bacteriorhodopsin-10-5", "bacteriorhodopsin-10-10", "br_corr_2", "bubble", "ubiquitin"]
specialNodes = [215, 215, 215, 210, -1, -1] # ggf. auch mit 216 als spez. Knoten
specialNodes2 = [216, 216, 216, 211, -1, -1] # ggf. auch mit 216 als spez. Knoten

# process all graphs in list
i = 0
for Gname in Gnames:
    G = readGraph("input/physchem/" + Gname + ".graph", Format.METIS)
    profile = profiling.Profile.create(G)
    profile.show()
    
    x = specialNodes[i]
    if (x >= 0):
        G = deactivateNode(G, x)
    partition(G, x, Gname)
    
    y = specialNodes2[i]
    if (y >= 0):
        G = deactivateNode(G, y)
    partition(G, y, Gname)    
    
    i = i+1